In [1]:
import sys
sys.path.append('/n/home12/binxuwang/Github/DiffusionReasoning')
sys.path.append("/n/home12/binxuwang/Github/DiT")
from diffusion import create_diffusion
from models import DiT
from stats_plot_utils import saveallforms, add_rectangles 
from repr_probe_lib import extract_features_DiT, train_pca_sgd_classifiers 
from circuit_toolkit.layer_hook_utils import print_specific_layer, get_module_name_shapes, featureFetcher_module

In [2]:
import os
from os.path import join
import torch 
import torch as th 
import torch.nn as nn
import pickle as pkl
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import einops
from torch.utils.data import Dataset, DataLoader, TensorDataset

In [3]:
train_attrs = np.load("/n/home12/binxuwang/Github/DiffusionReasoning/attr_all.npy")
train_attrs = th.from_numpy(train_attrs).to(int)
train_row_img = einops.rearrange(train_attrs, 'c s pnl (H W) att -> c s att H (pnl W)', H=3, W=3, att=3, pnl=3)
train_sample_img = einops.rearrange(train_row_img, 'c (S R) att H W -> c S att (R H) W', R=3,att=3, H=3, W=9)
labels_tsr = torch.arange(len(train_sample_img)).to(int).view(-1,1).repeat(1, train_sample_img.shape[1])

X_train = train_sample_img[:, :3000]
y_train = labels_tsr[:, :3000]
X_test = train_sample_img[:, 3000:]
y_test = labels_tsr[:, 3000:]
X_train = X_train.reshape(-1, 3, 9, 9)
y_train = y_train.reshape(-1)
X_test = X_test.reshape(-1, 3, 9, 9)
y_test = y_test.reshape(-1)
print(X_train.shape, y_train.shape)
train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)

torch.Size([120000, 3, 9, 9]) torch.Size([120000])


In [4]:
device = "cuda"

In [5]:
# abstract RAVEN dataset
dataset_Xmean = th.tensor([1.5, 2.5, 2.5]).view(1, 3, 1, 1).to(device)
dataset_Xstd = th.tensor([2.5, 3.5, 3.5]).view(1, 3, 1, 1).to(device)

In [6]:
DiT_configs = {
    "DiT_XL_1": {"depth": 28, "hidden_size": 1152, "patch_size": 1, "num_heads": 16},
    "DiT_XL_3": {"depth": 28, "hidden_size": 1152, "patch_size": 3, "num_heads": 16},
    "DiT_L_1": {"depth": 24, "hidden_size": 1024, "patch_size": 1, "num_heads": 16},
    "DiT_L_3": {"depth": 24, "hidden_size": 1024, "patch_size": 3, "num_heads": 16},
    "DiT_B_1": {"depth": 12, "hidden_size": 768, "patch_size": 1, "num_heads": 12},
    "DiT_B_3": {"depth": 12, "hidden_size": 768, "patch_size": 3, "num_heads": 12},
    "DiT_S_1": {"depth": 12, "hidden_size": 384, "patch_size": 1, "num_heads": 6},
    "DiT_S_3": {"depth": 12, "hidden_size": 384, "patch_size": 3, "num_heads": 6},
}
class_dropout_prob = 1.0
num_classes = 0
model_cfg = DiT_configs["DiT_S_1"]
model_DiT = DiT(input_size=9,
            in_channels=3, **model_cfg,
            mlp_ratio=4.0,
            class_dropout_prob=class_dropout_prob,
            num_classes=num_classes,
            learn_sigma=True,)

In [7]:
exproot = r"/n/holylfs06/LABS/kempner_fellow_binxuwang/Users/binxuwang/DL_Projects/DiT/results"
expname = r"090-RAVEN10_abstract-uncond-DiT_S_1-stream0_16M_heldout0_20240711-0204"
expdir = join(exproot, expname)
ckptdir = join(expdir, "checkpoints")
ckpt_path = join(ckptdir, "1000000.pt")
state_dict = th.load(ckpt_path, )
model_DiT.load_state_dict(state_dict["ema"])
model_DiT.to(device).eval();

In [8]:
t_scalar = 25
fetcher = featureFetcher_module()
for i in (5, 8, 11):#range(len(model.blocks)):
    fetcher.record_module(model_DiT.blocks[i], target_name=f"blocks{i}")
train_loader = DataLoader(train_dataset, batch_size=2048, shuffle=False, drop_last=False)
test_loader = DataLoader(test_dataset, batch_size=2048, shuffle=False)
feature_col = extract_features_DiT(model_DiT, fetcher, train_loader, dataset_Xmean, dataset_Xstd, t_scalar)
feature_col_test = extract_features_DiT(model_DiT, fetcher, test_loader, dataset_Xmean, dataset_Xstd, t_scalar)

  0%|          | 0/59 [00:00<?, ?it/s]

blocks5: torch.Size([120000, 81, 384])
blocks8: torch.Size([120000, 81, 384])
blocks11: torch.Size([120000, 81, 384])


  0%|          | 0/20 [00:00<?, ?it/s]

blocks5: torch.Size([40000, 81, 384])
blocks8: torch.Size([40000, 81, 384])
blocks11: torch.Size([40000, 81, 384])


In [9]:
feature_avg_col = {k: v.mean(dim=1) for k, v in feature_col.items()}
feature_avg_col_test = {k: v.mean(dim=1) for k, v in feature_col_test.items()}
feature_last_col = {k: v[:, -1] for k, v in feature_col.items()}
feature_last_col_test = {k: v[:, -1] for k, v in feature_col_test.items()}

In [10]:
!ls $STORE_DIR/DL_Projects

alexnet_rearing        GPT2_raven	     mini_edm
convrnn_Evol	       GPT-XL_grad_trace     resnet-sparsemax-model
CorNet-recurrent-evol  HaimDiffusionRNNProj  SiT
DiT		       insilico_exps	     VAR
edm_analy_traj_cmp     LLM_ft
edm_score_validation   Mamba_raven


In [11]:
!ls /n/holylfs06/LABS/kempner_fellow_binxuwang/Users/binxuwang/Datasets/RPM_dataset/RPM1000k

aR  attr_all_1000k.npy	attr_all_1000k.pt  attr_seq_tsr_all_1000k.pt


In [12]:
X_test.shape

torch.Size([40000, 3, 9, 9])

In [13]:
list(feature_avg_col)

['blocks5', 'blocks8', 'blocks11']

In [14]:
# ['blocks0', 'blocks2', 'blocks5', 'blocks8', 'blocks11']
feature_last_col_test['blocks11'].shape

torch.Size([40000, 384])

### Export for Shane

In [16]:
savedir = "/n/holylfs06/LABS/kempner_fellow_binxuwang/Users/binxuwang/Datasets/RPM_dataset/DiT_repr_data"
# feature_last_col_test
pkl.dump(feature_col_test, open(join(savedir, "feature_all_token_test.pkl"), "wb"))
pkl.dump(feature_avg_col_test, open(join(savedir, "feature_avg_token_test.pkl"), "wb"))
pkl.dump({"X": X_test, "y": y_test}, open(join(savedir, "X_y_test.pkl"), "wb"))

In [20]:
!du -Sh {savedir}/*

14G	/n/holylfs06/LABS/kempner_fellow_binxuwang/Users/binxuwang/Datasets/RPM_dataset/DiT_repr_data/feature_all_token_test.pkl
172M	/n/holylfs06/LABS/kempner_fellow_binxuwang/Users/binxuwang/Datasets/RPM_dataset/DiT_repr_data/feature_avg_token_test.pkl
72M	/n/holylfs06/LABS/kempner_fellow_binxuwang/Users/binxuwang/Datasets/RPM_dataset/DiT_repr_data/X_y_test.pkl


### Label shuffling classifier test

In [22]:
# shuffle y_train and y_test which are torch tensors
y_train_shuffled = y_train.clone()
y_test_shuffled = y_test.clone()
th.manual_seed(0)
y_train_shuffled = y_train[th.randperm(len(y_train))]
y_test_shuffled = y_test[th.randperm(len(y_test))]

In [23]:
model_PCA_col, PC_proj_col, results_col = train_pca_sgd_classifiers(
    feature_avg_col, feature_avg_col_test, y_train_shuffled, y_test_shuffled,
    PC_dim=1024, noPCA=True, num_classes=40, batch_size=None,
    num_epochs=5000, print_every=250, eval_every=1000, learning_rate=0.005,
    device='cuda'
)

Processing layer: blocks5
Epoch [1/5000], Loss: 3.8623, Accuracy: 0.0251
Epoch [250/5000], Loss: 3.6432, Accuracy: 0.0475
Epoch [500/5000], Loss: 3.6381, Accuracy: 0.0485
Epoch [750/5000], Loss: 3.6362, Accuracy: 0.0484
Epoch [1000/5000], Loss: 3.6351, Accuracy: 0.0493
Test Accuracy: 0.0241
Epoch [1250/5000], Loss: 3.6345, Accuracy: 0.0497
Epoch [1500/5000], Loss: 3.6339, Accuracy: 0.0501
Epoch [1750/5000], Loss: 3.6334, Accuracy: 0.0502
Epoch [2000/5000], Loss: 3.6332, Accuracy: 0.0500
Test Accuracy: 0.0241
Epoch [2250/5000], Loss: 3.6328, Accuracy: 0.0500
Epoch [2500/5000], Loss: 3.6325, Accuracy: 0.0499
Epoch [2750/5000], Loss: 3.6323, Accuracy: 0.0501
Epoch [3000/5000], Loss: 3.6320, Accuracy: 0.0502
Test Accuracy: 0.0242
Epoch [3250/5000], Loss: 3.6319, Accuracy: 0.0503
Epoch [3500/5000], Loss: 3.6318, Accuracy: 0.0502
Epoch [3750/5000], Loss: 3.6317, Accuracy: 0.0503
Epoch [4000/5000], Loss: 3.6315, Accuracy: 0.0503
Test Accuracy: 0.0243
Epoch [4250/5000], Loss: 3.6314, Accuracy: